In [148]:
import pandas as pd
import numpy as np
from pprint import pprint as pp
p = print

## Convert scrapy results to GTMS

TODO

## Load all needed static files

In [149]:
BASE_PATH = '../Raw data/'
FILENAMES = {
  'agencies': f'{BASE_PATH}agencies.csv'
}

# Base folder for GTFS datasets
FOLDERS = [
  'tt_extraurban',
  'tt_urban',
  'flixbus',
  'trenitalia/lombardia',
  'trenitalia/piemonte'
  # 'trenitalia/toscana'
]

GTFS_FILES = [
  'calendar',
  'calendar_dates',
  'routes',
  'stops',
  'stop_times',
  'trips'
] # , 'transfers'

# Trentino Trasporti
for folder in FOLDERS:
  for gtfs_file in GTFS_FILES:
    FILENAMES[f'{folder}_{gtfs_file}'] = f'{BASE_PATH}{folder}/{gtfs_file}.txt'

# Loading csv files
DFS = {}
for name, path in FILENAMES.items():
  DFS[name] = pd.read_csv(path)

FILENAMES

{'agencies': '../Raw data/agencies.csv',
 'tt_extraurban_calendar': '../Raw data/tt_extraurban/calendar.txt',
 'tt_extraurban_calendar_dates': '../Raw data/tt_extraurban/calendar_dates.txt',
 'tt_extraurban_routes': '../Raw data/tt_extraurban/routes.txt',
 'tt_extraurban_stops': '../Raw data/tt_extraurban/stops.txt',
 'tt_extraurban_stop_times': '../Raw data/tt_extraurban/stop_times.txt',
 'tt_extraurban_trips': '../Raw data/tt_extraurban/trips.txt',
 'tt_urban_calendar': '../Raw data/tt_urban/calendar.txt',
 'tt_urban_calendar_dates': '../Raw data/tt_urban/calendar_dates.txt',
 'tt_urban_routes': '../Raw data/tt_urban/routes.txt',
 'tt_urban_stops': '../Raw data/tt_urban/stops.txt',
 'tt_urban_stop_times': '../Raw data/tt_urban/stop_times.txt',
 'tt_urban_trips': '../Raw data/tt_urban/trips.txt',
 'flixbus_calendar': '../Raw data/flixbus/calendar.txt',
 'flixbus_calendar_dates': '../Raw data/flixbus/calendar_dates.txt',
 'flixbus_routes': '../Raw data/flixbus/routes.txt',
 'flixbus_st

## Generate Calendars

In [150]:
CALENDAR_COLS = ['weekdays', 'start_date', 'end_date']
calendars = pd.DataFrame(columns=CALENDAR_COLS)

for folder in FOLDERS:
  df = DFS[f'{folder}_calendar'].copy()
  weekdays = df.loc[:, 'monday':'sunday'].astype(str).agg(''.join, axis='columns')
  if len(weekdays) > 0:
    df['weekdays'] = weekdays
    df.rename(columns={'service_id': 'id'}, inplace=True)
    df.set_index('id', inplace=True)
    df.drop(columns=df.loc[:, 'monday':'sunday'].columns, inplace=True)
    # p(df)
    calendars = pd.concat([calendars, df])

if not calendars.index.is_unique:
  p('WARNING: duplicate id values!!')
  p(calendars.loc[calendars.index.duplicated()])

calendars.index.set_names('id', inplace=True)
calendars

,weekdays,start_date,end_date
id,,,
12023091120240621,1111111,20230911,20240621
12023091820240621,1111111,20230918,20240621
12023092820240621,1111111,20230928,20240621
12023100420240621,1111111,20231004,20240621
22023091120231015,1111110,20230911,20231015
...,...,...,...
249212715855674,1111111,20230728,20231231
249212915855675,1111111,20230728,20231231
249568015857568,0000011,20230728,20231231


### Calendar exceptions
The calendar_dates.txt table explicitly activates or disables service by date.

`exception_type` Enum	Required	Indicates whether service is available on the date specified in the date field. Valid options are:
- 1 - Service has been added for the specified date.
- 2 - Service has been removed for the specified date.

TODO: double check the meaning of service_id

In [151]:
CALENDAR_EXCEPTIONS_COLS = ['date', 'type']
calendar_exceptions = pd.DataFrame(columns=CALENDAR_EXCEPTIONS_COLS)

for folder in FOLDERS:
  df = DFS[f'{folder}_calendar_dates'].copy()
  # p(df)
  df.rename(columns={'service_id': 'id', 'exception_type': 'type'}, inplace=True)
  df.set_index('id', inplace=True)
  calendar_exceptions = pd.concat([calendar_exceptions, df])

# if not calendar_exceptions.index.is_unique:
#   p('WARNING: duplicate id values!!')
#   p(calendar_exceptions.loc[calendar_exceptions.index.duplicated()])

calendar_exceptions.index.set_names('id', inplace=True)
calendar_exceptions

,date,type
id,,
22023091120240621,20231101,2
22023091120240621,20231208,2
22023091120240621,20231225,2
22023091120240621,20231226,2
22023091120240621,20240101,2
...,...,...
ITRP-TS200_6501_63235_20221211_20231209,20231111,1
ITRP-TS200_6501_63235_20221211_20231209,20231118,1
ITRP-TS200_6501_63235_20221211_20231209,20231125,1


## Generate Bus Stops + Places

Notes:
- Extraurban has "wheelchair_boarding"

In [152]:
PLACES_COLS = ['address', 'latitude', 'longitude']
places = pd.DataFrame(columns=PLACES_COLS)

STOPS_COLS = ['name', 'type', 'place']
stops = pd.DataFrame(columns=STOPS_COLS)

for folder in FOLDERS:
  df = DFS[f'{folder}_stops'].copy()
  df['stop_id'] = df['stop_id'].astype(str)
  df.drop(columns=['stop_code', 'stop_desc', 'zone_id'], inplace=True, errors='ignore')
  df.rename(columns={'stop_id': 'id'}, inplace=True)
  df['place'] = df['id']
  df.set_index('id', inplace=True)

  stop_type = 'bus'
  if folder.startswith('trenitalia'):
    stop_type = 'train'
  df_stops = df[['stop_name', 'place']].copy()
  df_stops.rename(columns={'stop_name': 'name'}, inplace=True)
  df_stops = df_stops.assign(type=stop_type)

  df_places = df[['stop_lat', 'stop_lon']].copy()
  df_places.rename(columns={'stop_lat': 'latitude', 'stop_lon': 'longitude'}, inplace=True)
  # p(df, df_places, df_stops)
  
  places = pd.concat([places.astype(df_places.dtypes), df_places])
  stops = pd.concat([stops.astype(df_stops.dtypes), df_stops])

places.index.set_names('id', inplace=True)
stops.index.set_names('id', inplace=True)

stops

,name,type,place
id,,,
1,Trento-Autostaz.,bus,1
2,Trento Solteri Bivio Ss.12,bus,2
3,Trento Bren Center,bus,3
4,Lavis,bus,4
5,S.Michele A/Adige,bus,5
...,...,...,...
ITRP-ST0180849001,PIZZALE-LUNGAVILLA F.S.,train,ITRP-ST0180849001
ITRP-ST0150859001,CORBETTA S.STEF. F.S.,train,ITRP-ST0150859001
ITRP-ST0080559001,SAN REMO F.S.,train,ITRP-ST0080559001


In [153]:
places

,address,latitude,longitude
id,,,
1,NaN,46.070785,11.118457
2,NaN,46.084046,11.119728
3,NaN,46.091322,11.115660
4,NaN,46.138362,11.109135
5,NaN,46.193685,11.132520
...,...,...,...
ITRP-ST0180849001,NaN,45.049837,9.068991
ITRP-ST0150859001,NaN,45.481200,8.918130
ITRP-ST0080559001,NaN,43.825007,7.782201


## Generate Agencies

In [154]:
AGENCIES_COLS = ['name', 'url']

agencies = DFS['agencies'].copy()
agencies.rename(columns={'agency_id': 'id', 'agency_name': 'name', 'agency_url': 'url'}, inplace=True)
agencies.set_index('id', inplace=True)

agencies

,name,url
id,,
1,Trentino trasporti S.p.A.,https://www.trentinotrasporti.it/
2,Trenitalia S.p.A.,https://www.trentinotrasporti.it/
3,Flixbus,https://www.flixbus.it/


## Generate Routes

Could exctract `bikes_allowed`, `trip_bikes_allowed`!! (for now removed)

In [155]:
ROUTES_COLS = ['agency', 'short_name', 'long_name', 'type']
routes = pd.DataFrame(columns=ROUTES_COLS)

TRIPS_COLS = ['route', 'headsign', 'direction', 'calendar', 'accessibility']
trips = pd.DataFrame(columns=TRIPS_COLS)

TRIPS_STOPS_COLS = ['trip', 'arrival_time', 'departure_time', 'stop', 'stop_sequence']
trips_stops = pd.DataFrame(columns=TRIPS_STOPS_COLS)

for folder in FOLDERS:
  df_routes = DFS[f'{folder}_routes'].copy()
  # p(df['route_type'].value_counts())
  
  agency_id = 0
  route_type = 'bus'
  if folder.startswith('tt_'): # Trentino trasporti dataset
    # Filter only busses (exclude funivia serdagna and potentially others)
    df_routes = (df_routes.loc[df_routes['route_type'] == 3])
    agency_id = 1
  elif folder.startswith('trenitalia'):
    agency_id = 2
    route_type = 'train'
  elif folder.startswith('flixbus'):
    agency_id = 3
  else:
    raise 'Agency not found for folder: ' + folder
  
  # Assign agency and type
  df_routes = df_routes.assign(agency_id=agency_id)
  df_routes = df_routes.assign(type=route_type)

  df_routes.rename(columns={'route_id': 'id', 'agency_id': 'agency', 'route_short_name': 'short_name', 'route_long_name': 'long_name'}, inplace=True)
  df_routes.drop(columns=['route_type', 'route_color', 'route_text_color', 'route_desc', 'route_url', 'bikes_allowed', 'route_sort_order'], inplace=True, errors='ignore')
  df_routes.set_index('id', inplace=True)
  for k in ['short_name']:
    if k in df_routes:
      df_routes[k] = df_routes[k].astype(str)

  df_trips = DFS[f'{folder}_trips'].copy()

  df_trips.rename(columns={'trip_id': 'id', 'route_id': 'route', 'service_id': 'calendar', 'trip_headsign': 'headsign', 'direction_id': 'direction', 'wheelchair_accessible': 'accessibility'}, inplace=True)
  df_trips.drop(columns=['shape_id', 'trip_short_name', 'route_short_name', 'block_id', 'trip_bikes_allowed', 'bikes_allowed', 'train_category', 'ticketing_trip_id', 'ticketing_type'], inplace=True, errors='ignore')
  if 'accessibility' in df_trips:
    df_trips['accessibility'] = df_trips['accessibility'].apply(lambda x: '0' if np.isnan(x) else '1')
    df_trips['accessibility'] = df_trips['accessibility'].astype(str)
  else:
    df_trips = df_trips.assign(accessibility='NaN')
  df_trips.set_index('id', inplace=True)
  if 'headsign' in df_trips:
    df_trips['headsign'] = df_trips['headsign'].astype(str)
  if 'direction' in df_trips:
    df_trips['direction'] = df_trips['direction'].astype(float)
  df_trips['calendar'] = df_trips['calendar'].astype(str)

  df_trips_stops = DFS[f'{folder}_stop_times'].copy()
  for k in ['trip_id', 'stop_id']:
    if k in df_trips_stops:
      df_trips_stops[k] = df_trips_stops[k].astype(str)
  df_trips_stops.rename(columns={'trip_id': 'trip', 'stop_id': 'stop'}, inplace=True)
  df_trips_stops.drop(columns=['timepoint', 'stop_headsign', 'route_short_name', 'pickup_type', 'drop_off_type', 'shape_dist_traveled'], inplace=True, errors='ignore')

  routes = pd.concat([routes.astype(df_routes.dtypes), df_routes])
  trips = pd.concat([trips.astype(df_trips.dtypes), df_trips])
  trips_stops = pd.concat([trips_stops.astype(df_trips_stops.dtypes), df_trips_stops])

In [156]:
routes

,agency,short_name,long_name,type
1,1,B101,Cavalese-Predazzo-Moena-Canazei-Penia,bus
3,1,B102,Trento - Cembra - Capriana - Cavalese,bus
7,1,B114,Casatta di Valfloriana - Palu' - Sicina,bus
55,1,B104,Trento-S.Michele a/A-Salorno-Egna-Ora-Cavalese,bus
60,1,B105,Verla - Valternigo - Palu' di Giovo,bus
...,...,...,...,...
ITRP-RO200_6570_20221211_20231209,2,6570,VENTIMIGLIA - CUNEO,train
ITRP-RO200_6436_20221211_20231209,2,6436,VENTIMIGLIA - TORINO P.NUOVA,train
ITRP-RO200_6437_20221211_20231209,2,6437,VENTIMIGLIA - TORINO P.NUOVA,train
ITRP-RO200_6502_20221211_20231209,2,6502,VOGHERA - ALESSANDRIA,train


In [157]:
trips

,route,headsign,direction,calendar,accessibility
0003126722023091120240621,565,Sopramonte,0.0,22023091120240621,NaN
0003111702023091120240621,336,Peio,0.0,4512023091120240621,NaN
0003067862023091120240621,136,Tesero,0.0,52023091120240621,NaN
0003067032023091120240621,1,Vigo di Fassa-Strada Neva,1.0,302023091120240621,NaN
0003103112023091120240621,522,Castello Tesino,0.0,62023091120240621,NaN
...,...,...,...,...,...
ITRP-TR200_6501_65973,ITRP-RO200_6501_20221211_20231209,ASTI F.S.,NaN,ITRP-TS200_6501_65973_20230610_20231209,NaN
ITRP-TR200_6501_65975,ITRP-RO200_6501_20221211_20231209,ASTI F.S.,NaN,ITRP-TS200_6501_65975_20230610_20231209,NaN
ITRP-TR200_6501_65977,ITRP-RO200_6501_20221211_20231209,ASTI F.S.,NaN,ITRP-TS200_6501_65977_20230610_20231209,NaN
ITRP-TR200_6501_65978,ITRP-RO200_6501_20221211_20231209,ASTI F.S.,NaN,ITRP-TS200_6501_65978_20230610_20231209,NaN


In [158]:
trips_stops

,trip,arrival_time,departure_time,stop,stop_sequence
0,0003061822023091120240621,07:35:00,07:35:00,721,1
1,0003061822023091120240621,07:36:00,07:36:00,720,2
2,0003061822023091120240621,07:37:00,07:37:00,719,3
3,0003061822023091120240621,07:39:00,07:39:00,718,4
4,0003061822023091120240621,07:41:00,07:41:00,2377,5
...,...,...,...,...,...
9095,ITRP-TR200_6501_63235,14:24:00,14:25:00,ITRP-ST0061639001,7
9096,ITRP-TR200_6501_63235,14:29:00,14:30:00,ITRP-ST0060689001,8
9097,ITRP-TR200_6501_63235,14:35:00,14:36:00,ITRP-ST0050969001,9
9098,ITRP-TR200_6501_63235,14:39:00,14:40:00,ITRP-ST0050289001,10


# Data filtering

In future data filtering on the final datasets will be performed here.

# Save DataFrames to files

In [159]:
OUT_BASE_PATH = '../Parsed data/'
DATAFRAMES = {
  'agencies': agencies,
  'routes': routes,
  'trips': trips,
  'trip_calendars': calendars,
  'trip_calendar_exceptions': calendar_exceptions,
  'trip_stops': trips_stops,
  'stops': stops,
  'places': places
}

for name, df in DATAFRAMES.items():
  tmp = df.copy()
  cols = {}
  for c in df.columns:
    n = name[:-1]
    if name == 'agencies':
      n = 'agency'
    cols[c] = f'has_{n.capitalize()}_{c}'

  tmp = tmp.rename(columns=cols)
  tmp.index.set_names(f'has_{n.capitalize()}_id', inplace=True)

  # Write to file
  tmp.to_csv(f'{OUT_BASE_PATH}{name}.csv')